In [1]:
from __future__ import print_function, division
%pylab inline
import numpy as np
import pandas as pd
import seaborn as sns
from bokeh.plotting import *

from bokeh.io import output_notebook
output_notebook()
from collections import Counter
from operator import itemgetter

import sys
import bs4 as bs
import time
import dryscrape
import time
import json
from tqdm import tqdm
import urllib
from zipfile import ZipFile
import os
from itertools import groupby

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [2]:
df = pd.read_pickle('udacity_courses.df')

In [3]:
df.columns

Index([u'id', u'title', u'homepage', u'subtitle', u'level', u'starter',
       u'image', u'banner_image', u'teaser_video', u'summary',
       u'short_summary', u'required_knowledge', u'expected_learning',
       u'featured', u'syllabus', u'faq', u'full_course_available',
       u'expected_duration', u'expected_duration_unit', u'new_release',
       u'transcripts_url', u'transcripts_zip_fn', u'transcript'],
      dtype='object')

In [4]:
df['transcripts_zip_fn'].head(2)

0    transcripts_zips/bio110.zip
1     transcripts_zips/cs046.zip
Name: transcripts_zip_fn, dtype: object

In [5]:
import re
from collections import defaultdict
def extract_transripts_text(fn):
    if fn is None:
        return ('', '')
    lesson_dict = defaultdict(lambda : defaultdict(str))
    course = fn.rsplit('/', 1)[-1][:-4]
    with ZipFile(fn, 'r') as zf:
        files = [item for item in zf.namelist() if item.endswith('.srt')]
        for srt_f in files:
            if srt_f.startswith('Lesson'):
                title = srt_f.rsplit('/', 2)[-2]
                l_num = re.search("\d+", srt_f)
                if l_num:
                    l_num = int(l_num.group(0))
                    #print(l_num, end=' ')
                    #print(srt_f)
                    text = list()
                    with zf.open(srt_f) as f:
                        subs = parse_srt_content(f)
                        for s in subs:
                            text.extend(s.content)
                    text = ' '.join(text)
                    text = text.replace('>>', ' ')
                    while '  ' in text:
                        text = text.replace('  ', ' ')
                    lesson_dict[l_num]['title'] = title
                    lesson_dict[l_num]['text'] += ' ' + text
    return (course, dict(lesson_dict))

In [6]:
from collections import namedtuple
from itertools import groupby

# http://stackoverflow.com/a/23620587
def parse_srt_content(text):
    # "chunk" our input file, delimited by blank lines
    res = [list(g) for b, g in groupby(text, lambda x: bool(x.strip())) if b]

    Subtitle = namedtuple('Subtitle', 'number start end content')

    subs = []

    for sub in res:
        if len(sub) >= 3: # not strictly necessary, but better safe than sorry
            sub = [x.strip() for x in sub]
            # number, start_end, *content = sub # py3 syntax
            number = sub[0]
            start_end = sub[1]
            content = sub[2:]
            #print(start_end)
            start, end = start_end.split(' --> ')
            subs.append(Subtitle(number, start, end, content))
    return subs

In [7]:
r = list()
for zip_f in tqdm(df['transcripts_zip_fn'].values):
    l_dict = extract_transripts_text(zip_f)
    if len(l_dict[1]) > 1:
        r.append(l_dict)

100%|██████████| 129/129 [00:19<00:00,  6.57it/s]


In [8]:
#r[0]

In [10]:
data = list()
single_data = list()
for t, c in r:
    print(c.keys())
    lessons = c.keys()
    for key, val in c.items():
        single_data.append((t, key, val['title'], val['text']))
    sorted_lessons = sorted(map(int, lessons))
    for l1_num, l2_num in zip(sorted_lessons[:-1], sorted_lessons[1:]):
        l1 = c[l1_num]['text']
        l2 = c[l2_num]['text']
        data.append((l1, l2))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[1, 2, 3, 4, 5]
[0, 1, 2, 3]
[1, 2, 3, 4]
[1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[0, 1, 2, 3, 4, 5]
[1, 2, 3]
[1, 2, 3, 4, 5]
[8, 9, 10, 11, 12, 13, 14, 15, 16]
[1, 2, 3, 4, 5]
[0, 1, 2, 3]
[1, 2]
[1, 2]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 4]
[1, 2, 3, 4, 5]
[1, 2, 3]
[0, 1, 2, 3, 4, 5]
[1, 2]
[1, 2, 3]
[1, 2, 3, 4, 5]
[1, 2, 3]
[1, 2, 3]
[1, 2, 3, 4]
[1, 2

In [11]:
pd.DataFrame(columns=['course', 'lesson', 'title', 'text'], data=single_data).to_pickle('udacity_video_seq_simple.df')
print(len(data))

350


In [12]:
df = pd.DataFrame(columns=['v1', 'v2'], data=data)
neg_df = pd.DataFrame(columns=['v2', 'v1'], data=data)
df['target'] = 1.
neg_df['target'] = 0.
df = pd.concat([df, neg_df])

In [13]:
len(df)

700

In [14]:
df.head(2)

,target,v1,v2
0,1.0,"[SOUND]. Hey there, I'm Matt. One of the inst...",Let's get started with Tales from the Genome....
1,1.0,Let's get started with Tales from the Genome....,"Hey, Matt. Whatcha doin'? Oh, hey. Not much, ..."


In [15]:
df.to_pickle('udacity_videoseq.df')